# Pipeline de Processamento de Linguagem Natural

Passos que foram considerados para a realização da pipeline
---
Segmentação

Tokenização

Lematização

Stemming

Remoção de Stop Words

Rotulação de Part Of Speech

Reconhecimento de Entidades Nomeadas


---

## Preparação de ambiente para o algoritmo

In [1]:
pip install nltk

In [2]:
# Importação de lib
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
import re
import pandas as pd
import numpy as np

In [3]:
nltk.download('punkt')  # Baixa os dados necessários
nltk.download('wordnet')  # Baixa os dados para lematização
nltk.download('averaged_perceptron_tagger')  # Baixa os dados para rotulação de Part of Speech
nltk.download('maxent_ne_chunker')  # Baixa os dados para reconhecimento de Entidades Nomeadas
nltk.download('words')  # Baixa os dados adicionais
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Função universal - todos os passos do processamento

In [4]:
def process_nlp(texto):

    # Segmentação em frases
    frases = sent_tokenize(texto)

    # Tokenização e segmentação em palavras
    tokens_por_frase = [word_tokenize(frase) for frase in frases]

    # Inicializa o stemmer e lematizador
    stemmer = PorterStemmer()
    lematizador = WordNetLemmatizer()

    # Processamento de cada token
    tokens_processados = []

    for tokens_frase in tokens_por_frase:
        tokens_filtrados = [token.lower() for token in tokens_frase if token.isalpha()]

        # Stemming e lematização
        tokens_stemmed = [stemmer.stem(token) for token in tokens_filtrados]
        tokens_lematizados = [lematizador.lemmatize(token) for token in tokens_stemmed]

        # Remoção de stop words
        stopwords_en = set(stopwords.words('portuguese'))
        tokens_sem_stopwords = [token for token in tokens_lematizados if token not in stopwords_en]

        tokens_processados.append(tokens_sem_stopwords)

    # Rotulação de Part of Speech
    pos_tags = [pos_tag(tokens) for tokens in tokens_processados]

    # Reconhecimento de Entidades Nomeadas
    entidades_nomeadas = [ne_chunk(tagged_tokens) for tagged_tokens in pos_tags]

    return frases, tokens_processados, pos_tags, entidades_nomeadas

texto_exemplo = "A corrida aconteceu nos campos enquanto os cavalos corriam. João estava assistindo."
frases, tokens, pos_tags, entidades = process_nlp(texto_exemplo)

In [5]:
pos_tags

[[('corrida', 'NN'),
  ('aconteceu', 'NN'),
  ('campo', 'NN'),
  ('enquanto', 'IN'),
  ('cavalo', 'NN'),
  ('corriam', 'NN')],
 [('joão', 'NN'), ('assistindo', 'NN')]]

# Definição de funções unitárias

In [6]:
#Tokenização e segmentação em frases
def sentence_token(text):
  sentence = sent_tokenize(text)
  return sentence


# Tokenização e segmentação em palavras
def tokenize(sentences):
    words = []
    for sentence in sentences:
        w = word_extraction(sentence)
        words.extend(w)
    words = sorted(list(set(words)))
    return words

# Remoção de stopwords
def word_extraction(sentence):
    ignore = set(stopwords.words('portuguese'))
    words = re.sub("[^\w]", " ", sentence).split()
    cleaned_text = [w.lower() for w in words if w not in ignore]
    return cleaned_text

# Rotulação de Part of Speech
def pos_tags(tokens_process):
  #pos_tags = pos_tag(tokens_process)
  pos_tags = pos_tag(tokens_process)
  return pos_tags

# Reconhecimento de Entidades Nomeadas
def named_entities(pos_tags):
   named_entities = [ne_chunk(tagged_tokens) for tagged_tokens in pos_tags]
   return named_entities


## Pipeline

In [7]:
def pipeline(text):
  sentences = sentence_token(text)
  word = tokenize(sentence)
  classify = pos_tag(word)
  return classify


## Testes unitários

In [8]:
#Tokenização e segmentação em frases
# Input: "O rato roeu a roupa do rei de roma. Ferrari paz amor."
# Output esperado: ['O rato roeu a roupa do rei de roma.', 'Ferrari paz amor.']


texto = "O rato roeu a roupa do rato. Ferrari paz amor."
sentence = sentence_token(texto)
sentence

['O rato roeu a roupa do rato.', 'Ferrari paz amor.']

In [9]:
# Tokenização e segmentação em palavras
# Input: ['O rato roeu a roupa do rei de roma.', 'Ferrari paz amor.']
# Output esperado: ['amor', 'ferrari', 'o', 'paz', 'rato', 'roeu', 'roupa']

texto = sentence
word = tokenize(texto)
word

['amor', 'ferrari', 'o', 'paz', 'rato', 'roeu', 'roupa']

In [10]:
# Processamento dos tokens
# Input:['O rato roeu a roupa do rato.']
# Output esperado: ['o', 'rato', 'roeu', 'roupa', 'rato']

texto = sentence[0]
process = word_extraction(texto)
process

['o', 'rato', 'roeu', 'roupa', 'rato']

In [11]:
# Rotulação de Part of Speech
# Input: ['amor', 'ferrari', 'o', 'paz', 'rato', 'roeu', 'roupa']
# Output esperado: [('amor', 'NN'), ('ferrari', 'NN'), ('o', 'NN'), ('paz', 'NN'), ('rato', 'NN'), ('roeu', 'NN'), ('roupa', 'NN')]

texto = word
pos_tags = pos_tags(texto)
pos_tags

[('amor', 'NN'),
 ('ferrari', 'NN'),
 ('o', 'IN'),
 ('paz', 'NN'),
 ('rato', 'NN'),
 ('roeu', 'NN'),
 ('roupa', 'NN')]

## Teste de Pipeline

In [12]:
texto = "O rato roeu a roupa do rato"
pipeline(texto)

[('amor', 'NN'),
 ('ferrari', 'NN'),
 ('o', 'IN'),
 ('paz', 'NN'),
 ('rato', 'NN'),
 ('roeu', 'NN'),
 ('roupa', 'NN')]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Documentos processados a partir da pipeline
documentos = [
    "O rato roeu a roupa do rato.",
    "Ferrari paz amor.",
    "Outro documento de exemplo.",
    "Algo mais para processar."
]


# Importe o CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# Crie um objeto CountVectorizer
vectorizer = CountVectorizer()

# Processamento dos documentos e criação da matriz TF
documentos_processados = []

for doc in documentos:
    tokens_processados = pipeline(doc)  # Use sua função pipeline
    texto_processado = ' '.join([' '.join(tokens) for tokens in tokens_processados])
    documentos_processados.append(texto_processado)

matriz_tf = vectorizer.fit_transform(documentos_processados)

# A matriz TF é uma matriz esparsa, podemos convertê-la para um array NumPy
matriz_tf_array = matriz_tf.toarray()

# Obtenha os nomes das palavras (features) como colunas da matriz
nomes_palavras = vectorizer.get_feature_names_out()

# A matriz   contém as contagens de termos
print("Matriz de Frequência de Termos:")
print(matriz_tf_array)

# E os nomes das palavras correspondentes
print("\nNomes das Palavras:")
print(nomes_palavras)



Matriz de Frequência de Termos:
[[1 1 1 6 1 1 1 1]
 [1 1 1 6 1 1 1 1]
 [1 1 1 6 1 1 1 1]
 [1 1 1 6 1 1 1 1]]

Nomes das Palavras:
['amor' 'ferrari' 'in' 'nn' 'paz' 'rato' 'roeu' 'roupa']


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize, sent_tokenize

# Documentos processados a partir da pipeline
documentos = [
    "O rato roeu a roupa do rato.",
    "Ferrari paz amor.",
    "Outro documento de exemplo.",
    "Algo mais para processar."
]

# Funções auxiliares
def sentence_token(text):
    return sent_tokenize(text)

def tokenize(sentences):
    words = []
    for sentence in sentences:
        w = word_extraction(sentence)
        words.extend(w)
    words = sorted(list(set(words)))
    return words

def word_extraction(sentence):
    ignore = set(stopwords.words('portuguese'))
    words = re.sub("[^\w]", " ", sentence).split()
    cleaned_text = [w.lower() for w in words if w not in ignore]
    return cleaned_text

def pipeline(text):
    sentences = sentence_token(text)
    words = tokenize(sentences)
    return words

# Função principal
def process_documents(documentos):
    vectorizer = CountVectorizer()

    # Processamento dos documentos e criação da matriz TF
    documentos_processados = []

    for doc in documentos:
        tokens_processados = pipeline(doc)  # Use sua função pipeline
        texto_processado = ' '.join(tokens_processados)
        documentos_processados.append(texto_processado)

    matriz_tf = vectorizer.fit_transform(documentos_processados)

    # A matriz TF é uma matriz esparsa, podemos convertê-la para um array NumPy
    matriz_tf_array = matriz_tf.toarray()

    # Obtenha os nomes das palavras (features) como colunas da matriz
    nomes_palavras = vectorizer.get_feature_names_out()

    return matriz_tf_array, nomes_palavras

matriz_tf_array, nomes_palavras = process_documents(documentos)

# A matriz contém as contagens de termos
print("Matriz de Frequência de Termos:")
print(matriz_tf_array)

# E os nomes das palavras correspondentes
print("\nNomes das Palavras:")
print(nomes_palavras)


Matriz de Frequência de Termos:
[[0 0 0 0 0 0 0 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 0 0]
 [0 0 1 1 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0]]

Nomes das Palavras:
['algo' 'amor' 'documento' 'exemplo' 'ferrari' 'outro' 'paz' 'processar'
 'rato' 'roeu' 'roupa']


In [15]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!pip install flask-bootstrap
!ngrok authtoken '2QIX4LnKgVCm7rgxFme5TXTkJVZ_3scHFqy3ANZ8VHVqxAKPq'

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=c982d44986f24ed3b06c336a19db6094821e297387c2f9979d7c449d08854151
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460119 sha256=236a35a2722c9bb8ae240a9235636e76920ceb4e6a5aed0fd2708cfb96be98f7
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3925 sha256=18eae34b2f20de3120f802a0e41e9f2364df5bf2f98f5ee65654755587f56a0f
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d50

In [16]:
from flask import Flask, request, jsonify
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re

# flask_ngrok_example.py
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


app = Flask(__name__)

# Download das stopwords se ainda não tiver sido feito
nltk.download('stopwords')

# Documentos de exemplo
documentos = [
    "O rato roeu a roupa do rato.",
    "Ferrari paz amor.",
    "Outro documento de exemplo.",
    "Algo mais para processar."
]

# Definição de stopwords em português
stopwords_pt = set(stopwords.words('portuguese'))

# Crie um objeto CountVectorizer
vectorizer = CountVectorizer()

# Tokenização e segmentação em palavras
def word_extraction(text):
    words = re.sub("[^\w]", " ", text).split()
    cleaned_text = [w.lower() for w in words if w.lower() not in stopwords_pt]
    return ' '.join(cleaned_text)

# Processamento dos documentos e criação da matriz TF
documentos_processados = [word_extraction(doc) for doc in documentos]
matriz_tf = vectorizer.fit_transform(documentos_processados)
matriz_tf_array = matriz_tf.toarray()

# Obtenha os nomes das palavras (features) como colunas da matriz
nomes_palavras = vectorizer.get_feature_names_out()

# Função para processar o texto
def pipeline(text):
    # Vetorização do texto
    vetorizado = vectorizer.transform([text]).toarray()
    return {'matriz_tf_vetorizada': vetorizado.tolist()}

@app.route('/api/matriz_tf', methods=['POST'])
def obter_matriz_tf():
    if 'text' in request.json:
        texto = request.json['text']
        response = pipeline(texto)
        return jsonify(response)
    else:
        return jsonify({'error': 'Texto não encontrado na solicitação.'}), 400
if __name__ == '__main__':
    app.run()




 * Serving Flask app '__main__'
 * Debug mode: off


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
